In [3]:
import os
import numpy as np
import pandas as pd
import torch
import os.path as osp
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

DATA_DIR = '/home/chec/data/open-images/relation'

In [4]:
df_vrd = pd.read_csv(osp.join(DATA_DIR, 'challenge-2019-train-vrd.csv'))
df_vrd.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.00500,0.033125,0.580000,0.627778,0.00500,0.033125,0.580000,0.627778,is
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.61200,0.735000,0.418919,0.950450,0.61200,0.735000,0.418919,0.950450,is
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.37250,0.399375,0.706413,0.778557,0.37250,0.399375,0.706413,0.778557,is
3,4b6a08cc110d7275,/m/01mzpv,/m/01y9k5,0.17125,0.255625,0.557500,0.749167,0.20750,0.683125,0.611667,0.999167,at
4,0144cfbb726f4c72,/m/01mzpv,/m/04bcr3,0.85500,0.950000,0.561667,0.609167,0.82875,0.999375,0.568333,0.673333,at


In [32]:
df_vrd.shape

(374768, 13)

In [7]:
df_vrd.RelationshipLabel.value_counts()

is                194142
at                111493
on                 31604
holds              20986
plays               8932
interacts_with      3756
inside_of           2392
wears                836
hits                 593
under                 34
Name: RelationshipLabel, dtype: int64

In [33]:
df_box = pd.read_csv(osp.join(DATA_DIR, 'challenge-2019-train-vrd-bbox.csv'))
df_box.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
0,780d8a470d144fa0,/m/04yx4,0.000000,0.527143,0.029979,0.997859,0
1,780d8a470d144fa0,/m/04yx4,0.291429,0.687143,0.128480,0.997859,0
2,780d8a470d144fa0,/m/04yx4,0.590000,0.965714,0.164882,0.997859,0
3,780db0a7d07a4a20,/m/026t6,0.000000,1.000000,0.345625,0.990000,-1
4,780db0a7d07a4a20,/m/026t6,0.271667,0.626667,0.803125,0.999375,0


In [13]:
df_vrd['target'] = df_vrd.LabelName1.str.cat(df_vrd.LabelName2, sep=',')

In [53]:
df_hits = df_vrd.loc[df_vrd.RelationshipLabel=='hits'].copy()

In [20]:
df_hits.target.unique()

array(['/m/04yx4,/m/01226z', '/m/04yx4,/m/0wdt60w', '/m/01bl7v,/m/01226z',
       '/m/05r655,/m/01226z', '/m/03bt1vf,/m/01226z',
       '/m/04yx4,/m/05ctyq', '/m/03bt1vf,/m/05ctyq',
       '/m/05r655,/m/05ctyq'], dtype=object)

In [9]:
df_hits.LabelName2.unique()

array(['/m/01226z', '/m/0wdt60w', '/m/05ctyq'], dtype=object)

In [17]:
df_hits.LabelName1.unique()

array(['/m/04yx4', '/m/01bl7v', '/m/05r655', '/m/03bt1vf'], dtype=object)

In [19]:
classes = df_hits.target.unique().tolist() + df_hits.LabelName2.unique().tolist() + df_hits.LabelName1.unique().tolist()
classes

['/m/04yx4,/m/01226z',
 '/m/04yx4,/m/0wdt60w',
 '/m/01bl7v,/m/01226z',
 '/m/05r655,/m/01226z',
 '/m/03bt1vf,/m/01226z',
 '/m/04yx4,/m/05ctyq',
 '/m/03bt1vf,/m/05ctyq',
 '/m/05r655,/m/05ctyq',
 '/m/01226z',
 '/m/0wdt60w',
 '/m/05ctyq',
 '/m/04yx4',
 '/m/01bl7v',
 '/m/05r655',
 '/m/03bt1vf']

In [23]:
df_classes = pd.DataFrame({'class': classes})
df_classes.head()

,class
0,"/m/04yx4,/m/01226z"
1,"/m/04yx4,/m/0wdt60w"
2,"/m/01bl7v,/m/01226z"
3,"/m/05r655,/m/01226z"
4,"/m/03bt1vf,/m/01226z"


In [24]:
len(classes)

15

In [54]:
df_hits.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel,target
289,10b9cd603d074f3d,/m/04yx4,/m/01226z,0.696875,0.999375,0.331771,0.814433,0.720625,0.775625,0.645736,0.751640,hits,"/m/04yx4,/m/01226z"
1872,b954163b03180ddd,/m/04yx4,/m/0wdt60w,0.355222,0.641026,0.111632,0.999062,0.439024,0.548468,0.514071,0.653846,hits,"/m/04yx4,/m/0wdt60w"
2062,38a6c99644f16d9f,/m/01bl7v,/m/01226z,0.238000,0.527000,0.132500,0.953750,0.465000,0.570000,0.088750,0.212500,hits,"/m/01bl7v,/m/01226z"
2907,ac21b274af577a46,/m/01bl7v,/m/01226z,0.040541,0.556757,0.057762,0.971119,0.286486,0.472973,0.653430,0.891697,hits,"/m/01bl7v,/m/01226z"
3269,d39f87b7fb01e5ad,/m/05r655,/m/01226z,0.366940,0.807737,0.300781,0.759375,0.676436,0.781946,0.709375,0.780469,hits,"/m/05r655,/m/01226z"


In [55]:
df_hits = df_hits.drop(columns=['LabelName1', 'LabelName2', 'RelationshipLabel', 'XMin2', 'YMin2', 'XMax2', 'YMax2'], axis=1)
df_hits.head()

,ImageID,XMin1,XMax1,YMin1,YMax1,target
289,10b9cd603d074f3d,0.696875,0.999375,0.331771,0.814433,"/m/04yx4,/m/01226z"
1872,b954163b03180ddd,0.355222,0.641026,0.111632,0.999062,"/m/04yx4,/m/0wdt60w"
2062,38a6c99644f16d9f,0.238000,0.527000,0.132500,0.953750,"/m/01bl7v,/m/01226z"
2907,ac21b274af577a46,0.040541,0.556757,0.057762,0.971119,"/m/01bl7v,/m/01226z"
3269,d39f87b7fb01e5ad,0.366940,0.807737,0.300781,0.759375,"/m/05r655,/m/01226z"


In [56]:
df_hits.rename(columns={'XMin1': 'XMin', 'XMax1': 'XMax', 'YMin1': 'YMin', 'YMax1': 'YMax', 'target': 'LabelName'}, inplace=True)
df_hits.head()

,ImageID,XMin,XMax,YMin,YMax,LabelName
289,10b9cd603d074f3d,0.696875,0.999375,0.331771,0.814433,"/m/04yx4,/m/01226z"
1872,b954163b03180ddd,0.355222,0.641026,0.111632,0.999062,"/m/04yx4,/m/0wdt60w"
2062,38a6c99644f16d9f,0.238000,0.527000,0.132500,0.953750,"/m/01bl7v,/m/01226z"
2907,ac21b274af577a46,0.040541,0.556757,0.057762,0.971119,"/m/01bl7v,/m/01226z"
3269,d39f87b7fb01e5ad,0.366940,0.807737,0.300781,0.759375,"/m/05r655,/m/01226z"


In [29]:
df_box.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
0,8118f50240fef888,/m/01bl7v,0.412393,0.884615,0.003049,0.998476,0
1,811a624b0fd608b2,/m/0k4j,0.000000,0.903125,0.133333,0.937500,1
2,811a624b0fd608b2,/m/0k4j,0.534375,0.768750,0.191667,0.316667,0
3,811a624b0fd608b2,/m/0k4j,0.754687,1.000000,0.216667,0.370833,0
4,811a7ea31f6fdba6,/m/0k4j,0.157353,0.886765,0.328160,0.745011,0


In [34]:
df_box.shape

(3290070, 7)

In [35]:
len(df_box.ImageID.unique())

790478

In [36]:
img_hits = df_hits.ImageID.unique()

In [37]:
img_hits.shape

(520,)

In [39]:
df_human = df_box.loc[df_box.LabelName.isin(classes[11:])]
df_human_neg = df_human.loc[~df_human.ImageID.isin(set(img_hits))]
df_human_neg.shape

(2467302, 7)

In [40]:
img_human_neg = df_human_neg.ImageID.unique()[:1000]

In [41]:
df_football = df_box.loc[df_box.LabelName=='/m/01226z']
df_football.shape

(5097, 7)

In [44]:
img_football = df_football.ImageID.unique()[:1500]

In [45]:
img_football.shape

(1500,)

In [46]:
img_otherball = df_box.loc[df_box.LabelName.isin(['/m/0wdt60w', '/m/05ctyq'])].ImageID.unique()
img_otherball.shape

(655,)

In [48]:
selected_imgs = set(img_human_neg) | set(img_football) | set(img_otherball)

In [49]:
len(selected_imgs)

3135

In [66]:
df_box_hits = df_box.loc[df_box.ImageID.isin(selected_imgs)]
df_box_hits = df_box_hits.loc[df_box_hits.LabelName.isin(classes[8:])].copy()
df_box_hits.shape

(13181, 7)

In [51]:
df_box_hits = df_box_hits.drop(columns=['IsGroupOf'], axis=1)
df_box_hits.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,780d8a470d144fa0,/m/04yx4,0.000000,0.527143,0.029979,0.997859
1,780d8a470d144fa0,/m/04yx4,0.291429,0.687143,0.128480,0.997859
2,780d8a470d144fa0,/m/04yx4,0.590000,0.965714,0.164882,0.997859
9,780db1d52d5aee6c,/m/04yx4,0.441250,0.822500,0.365789,0.986842
10,780dc0ff229f14f1,/m/04yx4,0.722500,0.829375,0.758333,0.980000


In [58]:
df_meta = pd.concat([df_hits, df_box_hits], sort=False)
df_meta.sample(5)

,ImageID,XMin,XMax,YMin,YMax,LabelName
745329,5e8f6f703d23d938,0.376875,0.403750,0.085833,0.144167,/m/04yx4
87746,4644cd74d3efd717,0.521250,0.684375,0.411324,0.849292,"/m/04yx4,/m/01226z"
320653,b3e74b1511dc2316,0.365469,0.553024,0.093125,0.221250,/m/01226z
5499,ebdd002e8673f8e8,0.473125,0.592500,0.418919,0.717256,/m/01mzpv
2685,eb9a7891bed601c5,0.361250,0.420000,0.432343,0.528878,/m/04yx4


In [60]:
len(df_meta.ImageID.unique())

3464

In [61]:
df_meta.shape

(14512, 6)

In [62]:
df_box_hits.shape

(13919, 6)

In [63]:
df_hits.shape

(593, 6)